In [1]:
# load csv files and merge into 1 dataset
import os
import pandas as pd

moralis = pd.DataFrame({}, dtype=str)
filePath = "../data/Moralis/"
for filename in os.listdir(filePath):
    if not filename.endswith('csv'): continue # don't read non .csv files
    filePath_combined = filePath+filename
    newCsv = pd.read_csv(filePath_combined)
    moralis  = moralis.append(newCsv, ignore_index=True)


c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
def shortenInt(value):
    value = str(value)
    if len(value) > 7 : value = value[:-7]
    if value.isdigit(): return int(value)
    if value == "0.0" : return int(0)
    return print("ERROR: ", value)

In [3]:
moralis["value_cutDown"] = moralis["value"].map(lambda x : shortenInt(x))
moralis['value_cutDown'] = pd.to_numeric(moralis['value_cutDown'])
moralis['transactionEther'] = moralis["value_cutDown"] / 100000000000

moralis = moralis.drop(["value_cutDown"],axis=1)
moralis['transactionEther'] = pd.to_numeric(moralis['transactionEther'])

In [4]:
import pandas as pd
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("../DataOtherSources/EthereumNFTs/nfts.sqlite")

cur = con.cursor()
ethereum = pd.read_sql_query("SELECT * from transfers", con)
# print(df.head(10))
# Be sure to close the connection
con.close()

In [5]:
ethereum['transactionEther'] = ethereum["transaction_value"] / pow(10, 18)

In [6]:
import numpy as np
import functools

moralis = moralis[ moralis["contract_type"] == "ERC721"]

def disjunction(*conditions):
    return functools.reduce(np.logical_or, conditions)

block_max = ethereum["block_number"].max()
block_min = ethereum["block_number"].min()

c_1 = moralis["block_number"] > block_max
c_2 = moralis["block_number"] < block_min

moralis = moralis[disjunction(c_1,c_2)]

moralis = moralis.drop([  "block_hash",
                "block_number",
                "operator",
                "verified",
                "transaction_type",
                "transaction_index",
                "log_index",
                "contract_type",
                "from_address",
                "to_address",
                "amount",
                "value"
                ], axis=1)


In [7]:
ethereum = ethereum.drop([ 
                "block_number",
                "event_id", 
                "from_address",
                "to_address",
                "transaction_value"
                ], axis=1)

In [8]:
moralis.head(1)

,block_timestamp,transaction_hash,token_address,token_id,transactionEther
20732,2021-09-25T16:16:40.000Z,0xb32f5bb0ae464b781a3e4226589878d3be195b9db619...,0x2ea241b51451b713dd05566d10a6ba3379b26788,2959,0.0


In [9]:
moralis.columns = ['timestamp','transaction_hash','nft_address',"token_id","transactionEther"]

In [ ]:
ethereum.head(1)

In [10]:
# Sanity check
len(moralis.columns) == len(ethereum.columns) # expect true

True

In [ ]:
moralis['timestamp'] = pd.to_datetime(moralis['timestamp'])
# moralis['timestamp_day'] = moralis['timestamp'].dt.floor('d')

In [ ]:
ethereum['timestamp'] = pd.to_datetime(ethereum['timestamp'],unit='s')
# ethereum['timestamp_day'] = ethereum['timestamp'].dt.floor('d')

In [ ]:
merge = pd.concat([ethereum,moralis]).reset_index(drop=True)

In [ ]:
import datetime
x = datetime.datetime.now()
merge.to_csv('../data/merged/'+str(x.strftime("%Y%m%d"))+'merge.csv', index=False)